### 04 Embedding model（埋め込みモデル）と RAG（検索拡張生成）
gpt-oss:20bを使用する構成のため、**Colab GPU は L4 を使用すること。**
- 必要なライブラリをインストール
- Google Colab に Ollama をセットアップ
  - LLM モデルは gpt-oss:20b を使用（Ollama）
  - Embedding モデルは以下の2つから選択（どちらか一方のみ実行）
    - Option A: bge-m3（Ollama 経由）— 多言語対応
    - Option B: ruri-v3-310m（Sentence Transformers 経由）— 日本語特化・高精度
  - Reranker モデルは cl-nagoya/ruri-v3-reranker-310m を使用（Sentence Transformers）
- JAXA（宇宙航空研究開発機構）のリポジトリからデータをダウンロードして読み込み
> [井澤克彦, 市川信一郎, 高速回転ホイール: 高速回転ホイール開発を通しての知見, 宇宙航空研究開発機構研究開発報告, 2008](https://jaxa.repo.nii.ac.jp/records/2149)
- データの前処理
  - markdown に変換（MarkItDown を使用）
  - Unicode正規化 (NFKC), 1文字行ブロックの除去, 空行圧縮
  - チャンク分割
    - LangChain の SpacyTextSplitter を使用
    - spaCy の日本語モデルは、ja_ginza を使用
- ベクトルデータベースの構築（ChromaDB, インメモリ）
- 検索機能の実装と単体動作確認
  - キーワード検索 @ BM25（spaCyで形態素解析の前処理が必要）
  - Embedding model によるセマンティック検索
  - ハイブリッド検索
  - Reranker による再順位付け
- 検索機能をLLM の tool として定義
- 動作確認

**必要なライブラリをインストール**
- 1行にまとめることで pip が全パッケージの依存関係を一括解決する。
- 分割すると後勝ちで依存関係が壊れるリスクがある。
- NOTE: Colab では uv ではなく pip を使う。
> uv は依存解決の過程で numpy 等をアップグレードし、プリインストール済みの scipy 等を壊すため。

In [1]:
# Google Colab に必要なライブラリをインストールする。
# 1行にまとめることで pip が全パッケージの依存関係を一括解決する。
# NOTE: Colab では uv ではなく pip を使う。uv は依存解決の過程で
#       numpy 等をアップグレードし、プリインストール済みの scipy 等を壊すため。
# NOTE: langchain 関連は 1.x 系に明示的に指定する。
#       Colab プリインストールの 0.3.x が残ると langchain-mcp-adapters が動作しない。
%pip install -U ollama langchain-ollama \
     "langchain>=1.2.8" "langchain-core>=1.2.8" \
     "langgraph>=1.0.7" \
     "markitdown[all]" chromadb \
     "langchain-text-splitters>=0.3" \
     spacy ginza ja-ginza \
     rank-bm25 sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 86.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.8 MB/s eta 0:00:0000:010:01mm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 109.8 MB/s eta 0:00:0000:0100:01
   ━━━━

**Google Colab に Ollama をセットアップ**
- Ollama のインストール・起動・モデルのダウンロードを行う。
- 詳細は [01_connect_oss_llm.ipynb](01_connect_oss_llm.ipynb) を参照。

In [2]:
# Ollama のインストール・起動・モデルのダウンロード
# 詳細は 01_connect_oss_llm.ipynb を参照
import subprocess
import time
import ollama  # type: ignore

!apt-get install -y -qq zstd
!curl -fsSL https://ollama.com/install.sh | sh

process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
time.sleep(5)


def ollama_pull(model: str) -> None:
    """Ollama モデルをダウンロードし、進捗をインライン表示する。

    NOTE: ollama pull のプログレスバーは Colab で文字化けするため、
          Python API 経由でステータスのみ表示する。
    """
    for progress in ollama.pull(model, stream=True):
        status = progress.get("status", "")
        total = progress.get("total") or 0
        completed = progress.get("completed") or 0
        if total:
            line = f"{status}: {completed / total:.0%}"
        else:
            line = status
        print(f"\r{line:<60}", end="", flush=True)
    print(f"\n{model}: Done!")


# AI エージェントにはツールコール対応モデルが必要。
model_name = "gpt-oss:20b"
ollama_pull(model_name)
!ollama show {model_name}

Selecting previously unselected package zstd.
(Reading database ... 121852 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
success                                                     
gpt-oss:20b: Done!
  Model
    architecture        gptoss    
    parameters          20.9B     
    context length      131072    
    embedding length    2880      
    quantization        MXFP4     

  Ca

**ChatOllama で LLM に接続**
- 詳細は [01_connect_oss_llm.ipynb](01_connect_oss_llm.ipynb) を参照。

In [3]:
# ChatOllama で LLM に接続する。
from langchain_ollama import ChatOllama  # type: ignore

llm = ChatOllama(
    model="gpt-oss:20b",
    num_ctx=16384,
    num_predict=-1,
    temperature=0.8,
    top_k=40,
    top_p=0.9,
    repeat_penalty=1.1,
    reasoning=None,
)

**Embedding モデル（bge-m3）をダウンロード**
- bge-m3 は多言語対応の Embedding モデル。日本語にも対応。
- Ollama 経由で利用する。

In [ ]:
# Embedding モデル (bge-m3) をダウンロードする。
embedding_model_name = "bge-m3"
ollama_pull(embedding_model_name)
!ollama show {embedding_model_name}

**Embedding モデルと Reranker モデルのセットアップ**

Embedding モデルは **以下の2つから選択**して実行すること（どちらか一方のみ実行）。
- **Option A: bge-m3**（Ollama 経由） — 多言語対応。Ollama でインフラ統一。
- **Option B: ruri-v3-310m**（Sentence Transformers 経由） — 日本語特化。JMTEB 2位。

| 項目 | bge-m3 (Option A) | ruri-v3-310m (Option B) |
|------|-------------------|------------------------|
| JMTEB 平均 | 72.46（10位） | 75.85（2位） |
| パラメータ | 567M | 310M |
| Embedding 次元 | 1024 | 768 |
| バックエンド | Ollama | Sentence Transformers (PyTorch) |

Reranker は cl-nagoya/ruri-v3-reranker-310m を使用（どちらの Embedding でも共通）。

**Embedding モデル / Reranker モデルの性能は、RAG の性能に直結するので、日本語性能が高いモデルを選定するのが基本。**

Embedding モデル
> [JMTEBリーダーボード](https://github.com/sbintuitions/JMTEB/blob/main/leaderboard.md)

Reranker モデル
> [ruri-v3-reranker のモデルカード内に記載されるランキング](https://huggingface.co/cl-nagoya/ruri-v3-reranker-310m)

In [ ]:
# === Option A: bge-m3 (Ollama 経由) ===
# Ollama でインフラを統一したい場合はこちらを実行する。
# NOTE: Option B と排他。どちらか一方のみ実行すること。
from langchain_ollama import OllamaEmbeddings  # type: ignore

embeddings = OllamaEmbeddings(model=embedding_model_name)

# 動作確認: 短いテキストを埋め込んでベクトル次元を確認する。
test_vec = embeddings.embed_query("テスト文です")
print(f"[Option A] bge-m3 — Embedding dim: {len(test_vec)}")

ruri-v3-310m の場合は、モデルの仕様として、プレフィックスを付ける必要がある。

```
# Ruri v3 employs a 1+3 prefix scheme to distinguish between different types of text inputs:
# "" (empty string) is used for encoding semantic meaning.
# "トピック: " is used for classification, clustering, and encoding topical information.
# "検索クエリ: " is used for queries in retrieval tasks.
# "検索文書: " is used for documents to be retrieved.
sentences = [
    "川べりでサーフボードを持った人たちがいます",
    "サーファーたちが川べりに立っています",
    "トピック: 瑠璃色のサーファー",
    "検索クエリ: 瑠璃色はどんな色？",
    "検索文書: 瑠璃色（るりいろ）は、紫みを帯びた濃い青。名は、半貴石の瑠璃（ラピスラズリ、英: lapis lazuli）による。JIS慣用色名では「こい紫みの青」（略号 dp-pB）と定義している[1][2]。",
]
```
> [cl-nagoya/ruri-v3-310m のモデルカード](https://huggingface.co/cl-nagoya/ruri-v3-310m)

**ruri-v3-310m の LangChain ラッパーの解説**
1. `class RuriEmbeddings(Embeddings)` で、LangChain の `Embeddings` クラスを継承。（LangChain の機能を、ラッパー関数でも使えるようにする。）
2. `SentenceTransformer(model_name)` でモデルを読み込み。
3. `prefixed = [f"検索文書: {t}" for t in texts]` で、検索対象のテキストの冒頭に、`検索文書: `を追加する。その後、`.tolist()` で、list[float] 型（LangChain のインターフェースとなる型）に変換する。
4. 同様に、`self.model.encode(f"検索クエリ: {text}").tolist()` で、`検索クエリ: ` のプレフィックスを追加する。

In [4]:
# === Option B: ruri-v3-310m (Sentence Transformers 経由) ===
# 日本語特化で高精度な Embedding を使いたい場合はこちらを実行する。
# NOTE: Option A と排他。どちらか一方のみ実行すること。
from langchain_core.embeddings import Embeddings  # type: ignore
from sentence_transformers import SentenceTransformer  # type: ignore


class RuriEmbeddings(Embeddings):
    """ruri-v3 を LangChain の Embeddings インターフェースでラップする。

    ruri-v3 は query と document で異なるプレフィックスが必要：
    - query: "検索クエリ: "
    - document: "検索文書: "
    """

    def __init__(self, model_name: str = "cl-nagoya/ruri-v3-310m") -> None:
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        prefixed = [f"検索文書: {t}" for t in texts]
        return self.model.encode(prefixed).tolist()

    def embed_query(self, text: str) -> list[float]:
        return self.model.encode(f"検索クエリ: {text}").tolist()


embeddings = RuriEmbeddings()

# 動作確認: 短いテキストを埋め込んでベクトル次元を確認する。
test_vec = embeddings.embed_query("テスト文です")
print(f"[Option B] ruri-v3-310m — Embedding dim: {len(test_vec)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

[Option B] ruri-v3-310m — Embedding dim: 768


In [5]:
# Reranker: Sentence Transformers の CrossEncoder を使用する。
# cl-nagoya/ruri-v3-reranker-310m: 日本語特化の高精度 Reranker (ModernBERT-Ja ベース)。
# 初回実行時にモデルがダウンロードされる。
from sentence_transformers import CrossEncoder  # type: ignore

reranker = CrossEncoder("cl-nagoya/ruri-v3-reranker-310m")
print("Reranker model loaded.")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/156 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Reranker model loaded.


**JAXA（宇宙航空研究開発機構）のリポジトリからデータをダウンロードして読み込み**

> [井澤克彦, 市川信一郎, 高速回転ホイール: 高速回転ホイール開発を通しての知見, 宇宙航空研究開発機構研究開発報告, 2008](https://jaxa.repo.nii.ac.jp/records/2149)

In [6]:
# JAXA リポジトリから PDF をダウンロードし、MarkItDown で markdown に変換する。
import urllib.request
from pathlib import Path
from markitdown import MarkItDown  # type: ignore

pdf_url = "https://jaxa.repo.nii.ac.jp/record/2149/files/63826000.pdf"
pdf_path = Path("高速回転ホイール.pdf")

if not pdf_path.exists():
    urllib.request.urlretrieve(pdf_url, pdf_path)
    print(f"ダウンロード完了: {pdf_path}")

md = MarkItDown()
result = md.convert(str(pdf_path))
raw_text = result.text_content

print(f"文字数: {len(raw_text)}")
print("--- 先頭 500 文字 ---")
print(raw_text[:500])

ダウンロード完了: 高速回転ホイール.pdf
文字数: 86888
--- 先頭 500 文字 ---
This document is provided by JAXA.This document is provided by JAXA.This document is provided by JAXA.  目  次

1.  はじめに ································································································ 2

2.  宇宙用フライホイールの概要 ····································································· 3

2.1.    動作原理 ··························································································· 3

2.2.    システム構成 ································································


**データの前処理**
- Unicode 正規化 (NFKC)
  - 全角/半角などの表記を統一する。（例えば、スペースの全角は、半角に変換される。）
- テキストクリーニング（PDF抽出時のレイアウト崩れ対策）
  - 1文字行が連続するブロックを除去（図表・縦書き由来のゴミ）
  - 連続する空行を圧縮
- チャンク分割（SpacyTextSplitter + ja_ginza）

In [7]:
# Unicode 正規化 (NFKC) を適用する。
# 全角英数→半角、半角カナ→全角 などを統一する。
import unicodedata

text = unicodedata.normalize("NFKC", raw_text)
print(f"正規化後の文字数: {len(text)}")

正規化後の文字数: 86912


**補足：正規表現**
このコードの正規表現は、それぞれ以下を意味している。
- `^` ：行の先頭
- `[^\S\n]*` ：改行以外の空白が 0 個以上（ここの `^` は ～ でない）
- `\S` ：空白以外の文字がちょうど1つ
- `$` ：行末
- `\n` ：改行
- `?` ：～ があるかもしれないし、ないかもしれない
- `{3,}` ：3つ以上連続している
> `(^ [^\S\n]* \S [^\S\n]* $ \n?) {3,}` ：行頭から始まり、前後にスペースがあっても良いが実質1文字しかなく、行末（と改行）で終わる」という行が、3回以上連続した場合という意味になる。

In [8]:
# PDF 抽出テキストの汎用クリーニング。
# 図表や縦書きからの抽出で 1 文字ずつ改行されたゴミテキストを除去し、
# 連続する空行を圧縮する。
import re


def clean_pdf_text(text: str) -> str:
    """PDF 抽出テキストの汎用クリーニングを行う。

    1. 1文字行が 3 行以上連続するブロックを除去（図表・縦書き由来のゴミ）
    2. 連続する空行を 2 つまでに圧縮
    """
    # 1文字行（空白除く）が 3 行以上連続するブロックを除去する。
    # 例: "D\n\nW\n)\nU\n..." のようなパターン
    text = re.sub(
        r"(^[^\S\n]*\S[^\S\n]*$\n?){3,}",
        "\n",
        text,
        flags=re.MULTILINE,
    )
    # 連続する空行を 2 つまでに圧縮する。
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


text = clean_pdf_text(text)
print(f"クリーニング後の文字数: {len(text)}")
print("--- 先頭 500 文字 ---")
print(text[:500])

クリーニング後の文字数: 79515
--- 先頭 500 文字 ---
This document is provided by JAXA.This document is provided by JAXA.This document is provided by JAXA.  目  次

1.  はじめに ································································································ 2

2.  宇宙用フライホイールの概要 ····································································· 3

2.1.    動作原理 ··························································································· 3

2.2.    システム構成 ································································


Sudachi（spaCy の形態素解析器）の入力上限 (49,149 bytes) を超えないように、テキストを段落区切りで事前分割してから SpacyTextSplitter に渡す。

**ブロック分割処理の補足**
- `candidate = current + "\n\n" + para if current else para` の、`if current else para` は三項演算子で、`current` が空なら `para` を、空でなければ `current + "\n\n" + para` を実行するという処理。（最初の処理では、`current` が空なので、これをやらないと、無意味な改行ができる。）
- `if len(candidate.encode("utf-8")) > max_bytes and current:
            blocks.append(current)
            current = current[-overlap_chars:] + "\n\n" + para`

> `candidate` が入力上限を超えた場合、今の `current` を 1ブロックとして、次のブロック作成に移る。このとき、`current` の初期値を、`overlap_chars` 分、前の文章と重複させる。これにより、ブロック分割によって、文脈が途切れることを防いでいる。

In [9]:
# SpacyTextSplitter でチャンク分割する。
# spaCy の日本語モデル ja_ginza を使用し、文境界を考慮して分割する。
# NOTE: Sudachi の入力上限 (49,149 bytes) を超えないように、
#       テキストを段落区切りで事前分割してから SpacyTextSplitter に渡す。
from langchain_text_splitters import SpacyTextSplitter  # type: ignore

CHUNK_SIZE = 1500
CHUNK_OVERLAP = 300
BLOCK_MAX_BYTES = 40_000  # Sudachi 上限 (49,149) に対する安全マージン
BLOCK_OVERLAP_CHARS = CHUNK_SIZE  # ブロック境界でも 1 チャンク分の重複を保証


def split_into_safe_blocks(
    text: str,
    max_bytes: int = BLOCK_MAX_BYTES,
    overlap_chars: int = BLOCK_OVERLAP_CHARS,
) -> list[str]:
    """テキストを段落区切りで max_bytes 以下のブロックに分割する。

    ブロック間で overlap_chars 分の重複を持たせることで、
    ブロック境界をまたぐ文脈がチャンク分割で失われないようにする。
    """
    paragraphs = text.split("\n\n")
    blocks: list[str] = []
    current = ""
    for para in paragraphs:
        candidate = current + "\n\n" + para if current else para
        if len(candidate.encode("utf-8")) > max_bytes and current:
            blocks.append(current)
            current = current[-overlap_chars:] + "\n\n" + para
        else:
            current = candidate
    if current:
        blocks.append(current)
    return blocks


text_splitter = SpacyTextSplitter(
    separator="\n\n",  # 段落区切りで結合（ブロック分割と同じ区切り文字）
    pipeline="ja_ginza",
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

blocks = split_into_safe_blocks(text)
print(f"事前分割ブロック数: {len(blocks)}")

chunks: list[str] = []
for block in blocks:
    chunks.extend(text_splitter.split_text(block))

print(f"チャンク数: {len(chunks)}")
for i, chunk in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i} ({len(chunk)} chars) ---")
    print(chunk[:200] + "..." if len(chunk) > 200 else chunk)

事前分割ブロック数: 5
チャンク数: 76

--- Chunk 0 (1326 chars) ---
This document is provided by JAXA.This document is provided by JAXA.This document is provided by JAXA.  目  

次

1.  

はじめに ··········································································...

--- Chunk 1 (1487 chars) ---
宇宙用フライホイール仕様緒元と使用に当たっての注意事項 ·························· 49

4.1.    仕様緒元の見方 ················································································ 49

4

.2.    使用に当たっての調整

事項/注意事項 ··········...

--- Chunk 2 (1482 chars) ---
low

 disturbance  characteristics  

since  

FY2001.  

The

advanced  reaction  wheel  

is  

characterized  

by  

small  

and  high  

performance  in

comparison  with  a  current  

domestic...


**ベクトルデータベースの構築（ChromaDB, インメモリ）**

チャンクを Embedding してベクトルデータベースに格納する。
- `chroma_client = chromadb.Client()` ：データベースの起動（インメモリ）
- `collection = chroma_client.create_collection(name="jaxa_wheel", metadata={"hnsw:space": "cosine"},)`
> `name` でデータベースの名前を、`hnsw:space` で検索方法を指定。（ここでは、RAG で一般的に使用されるコサイン類似度を設定。）
- `embeddings.embed_documents(chunks)` ：チャンクを Embedding する。（テキスト情報を意味のある数値情報に変換する処理。）
- `collection.add` ：各チャンクの id番号、オリジナルのテキスト情報、Embedding 結果をベクトルデータベースに保存する。

In [10]:
# ChromaDB にチャンクを格納する（インメモリ）。
import chromadb  # type: ignore

chroma_client = chromadb.Client()

collection = chroma_client.create_collection(
    name="jaxa_wheel",
    metadata={"hnsw:space": "cosine"},
)

# チャンクを Embedding してデータベースに追加する。
chunk_embeddings = embeddings.embed_documents(chunks)

collection.add(
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    documents=chunks,
    embeddings=chunk_embeddings,
)

print(f"ChromaDB に {collection.count()} 件のチャンクを格納しました。")

ChromaDB に 76 件のチャンクを格納しました。


**検索機能の実装**
- キーワード検索 @ BM25（spaCy で形態素解析の前処理が必要）
- Embedding model によるセマンティック検索
- ハイブリッド検索
- Reranker による再順位付け

**キーワード検索（BM25）の検索インデックスの作成**

BM25 は、単語を抽出して検索インデックスを作成する、英語などの言語向けのアルゴリズム。しかし、日本語の場合では、英語（I like an apple.）のような単語間の空白がないので、形態素解析を行い、単語の抽出を行う必要がある。
- 日本語の辞書は、`ja_ginza` を使用する。（`nlp = spacy.load("ja_ginza", disable=["parser", "ner"])`
- 形態素解析には、spaCy を使用する。抽出する単語は、名詞 (NOUN), 動詞 (VERB), 形容詞 (ADJ), 固有名詞 (PROPN), 数値 (NUM) とする。（しかし、みたいな接続詞などは、キーワード検索には不要なので抽出しない。）
> `if token.pos_ not in include_pos:` ：`include_pos = {"NOUN", "VERB", "ADJ", "PROPN", "NUM"}` に含まれない単語を読み飛ばす処理をしている。
- `if token.is_stop:` ：ストップワード（"それ" などの意味のない代名詞や、"する" などの汎用動詞）を、日本語の辞書（`ja_ginza`）を使って読み飛ばす処理をしている。
- `lemma = token.lemma_` ：走った」「走る」などをすべて「走る」という基本形に統一し、表記ゆれを削減する。（レンマタイゼーションという。）
- `if len(lemma) == 1 and re.match(r"[ぁ-ん\u30fc!-/:-@\[-`{-~]", lemma):` ：意味のない 1文字の単語を読み飛ばす処理をしている。（正規表現で、ひらがなと、各種記号を指定している。）
> 軸、など、1文字でも意味のある単語は残る指示になっている。

In [11]:
# BM25 用の前処理: spaCy (ja_ginza) で形態素解析してトークン化する。
import re  # noqa: F811
import spacy  # type: ignore
from rank_bm25 import BM25Okapi  # type: ignore

# 処理速度向上のため、不要なコンポーネントを無効化してロード
# （トークナイズとレンマ化だけなら parser, ner は不要）
nlp = spacy.load("ja_ginza", disable=["parser", "ner"])


def tokenize(text: str) -> list[str]:
    """spaCy で形態素解析し、BM25 用のトークンリストを返す。"""
    doc = nlp(text)
    tokens = []

    # 含める品詞（名詞 (NOUN), 動詞 (VERB), 形容詞 (ADJ), 固有名詞 (PROPN), 数値 (NUM)）
    include_pos = {"NOUN", "VERB", "ADJ", "PROPN", "NUM"}

    for token in doc:
        if token.pos_ not in include_pos:
            continue
        if token.is_stop:
            continue

        lemma = token.lemma_

        # 1文字トークンの処理:
        # ひらがな・記号は除外するが、漢字・カタカナ・英数字は残す。
        # 例: 「は」「が」→ 除外、「目」「軸」「C」「5」→ 残す
        if len(lemma) == 1 and re.match(r"[ぁ-ん\u30fc!-/:-@\[-`{-~]", lemma):
            continue

        tokens.append(lemma)

    return tokens


# チャンクをトークン化して BM25 インデックスを構築する。
tokenized_chunks = [tokenize(chunk) for chunk in chunks]
bm25 = BM25Okapi(tokenized_chunks)

print(f"BM25 インデックス構築完了: {len(tokenized_chunks)} 件")
print(f"トークン例（先頭チャンク）: {tokenized_chunks[0][:10]}")

BM25 インデックス構築完了: 76 件
トークン例（先頭チャンク）: ['This', 'Document', 'is', 'provided', 'by', 'JAXA', 'This', 'Document', 'is', 'provided']


**2段階の RAG 検索の実装（Retrieval & Rerank）**
- 1段目では、ハイブリッド検索（キーワード検索（BM25）＋セマンティック検索の合計スコアによる検索）によって、広く検索をする。スコアの合計には、Reciprocal Rank Fusion（RRF）を使用する。
- 2段目では、Reranker モデルを使って、1段目の検索結果を、質問に対して意味合いが近い順に、高精度な再順位付けをする。


<u>RAG 検索のパラメータ<u/>
| 項目 | デフォルト値 | 備考 |
|:---|:---:|:---|
| RETRIEVAL_TOP_K | 20 | 1段目の検索数。広くとる。 |
| RERANK_TOP_K | 5 | 2段目の検索数。＝ LLM に渡す情報数。 |
| BM25_WEIGHT | 0.3 | ハイブリッド検索におけるキーワード検索の重み（1に近づくほど大きい） |
| k（固定値） | 60 | ハイブリッド検索におけるパラメータ。通常は調整しない。 |

In [12]:
import numpy as np  # type: ignore

RETRIEVAL_TOP_K = 20  # 第1段検索（BM25 / セマンティック / ハイブリッド）の抽出数
RERANK_TOP_K = 5  # 第2段検索（Reranker）の抽出数
BM25_WEIGHT = 0.3  # ハイブリッド検索における BM25 の重み（残りがセマンティック）


def search_bm25(query: str, top_k: int = RETRIEVAL_TOP_K) -> list[dict]:
    """BM25 によるキーワード検索を行う。"""
    tokenized_query = tokenize(query)
    scores = bm25.get_scores(tokenized_query)
    top_indices = np.argsort(scores)[::-1][:top_k]
    return [
        {
            "rank": rank + 1,
            "chunk_id": int(idx),
            "score": float(scores[idx]),
            "text": chunks[idx],
        }
        for rank, idx in enumerate(top_indices)
        if scores[idx] > 0
    ]


def search_semantic(query: str, top_k: int = RETRIEVAL_TOP_K) -> list[dict]:
    """Embedding model によるセマンティック検索を行う。"""
    query_embedding = embeddings.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
    )
    return [
        {
            "rank": rank + 1,
            "chunk_id": int(
                doc_id.split("_")[1]
            ),  # "_" はベクトルデータベース定義時の ids の 区切り文字に合わせること
            "score": 1.0 - dist,  # 距離（dist）をスコア（max 1.0）に変換
            "text": doc,
        }
        for rank, (doc_id, doc, dist) in enumerate(
            zip(results["ids"][0], results["documents"][0], results["distances"][0])
        )
    ]


def search_hybrid(
    query: str, top_k: int = RETRIEVAL_TOP_K, bm25_weight: float = BM25_WEIGHT
) -> list[dict]:
    """BM25 とセマンティック検索のハイブリッド検索を行う。

    Reciprocal Rank Fusion (RRF) でスコアを統合する。
    """
    k = 60  # RRF のハイパーパラメータ

    bm25_results = search_bm25(query, top_k=top_k)
    semantic_results = search_semantic(query, top_k=top_k)

    # RRF スコアを計算する。
    scores: dict[int, float] = {}
    texts: dict[int, str] = {}

    for r in bm25_results:
        cid = r["chunk_id"]
        scores[cid] = scores.get(cid, 0) + bm25_weight / (k + r["rank"])
        texts[cid] = r["text"]

    semantic_weight = 1.0 - bm25_weight
    for r in semantic_results:
        cid = r["chunk_id"]
        scores[cid] = scores.get(cid, 0) + semantic_weight / (k + r["rank"])
        texts[cid] = r["text"]

    sorted_ids = sorted(scores, key=lambda cid: scores[cid], reverse=True)[:top_k]
    return [
        {"rank": rank + 1, "chunk_id": cid, "score": scores[cid], "text": texts[cid]}
        for rank, cid in enumerate(sorted_ids)
    ]


def rerank(query: str, results: list[dict], top_k: int = RERANK_TOP_K) -> list[dict]:
    """Reranker (CrossEncoder) で検索結果を再順位付けする。"""
    if not results:
        return []
    pairs = [(query, r["text"]) for r in results]
    scores = reranker.predict(pairs)
    ranked_indices = np.argsort(scores)[::-1][:top_k]
    return [
        {
            "rank": rank + 1,
            "chunk_id": results[idx]["chunk_id"],
            "score": float(scores[idx]),
            "text": results[idx]["text"],
        }
        for rank, idx in enumerate(ranked_indices)
    ]


print("検索関数を定義しました: search_bm25, search_semantic, search_hybrid, rerank")
print(f"  第1段検索 top_k: {RETRIEVAL_TOP_K}, 第2段 Rerank top_k: {RERANK_TOP_K}")
print(f"  BM25 weight: {BM25_WEIGHT}, Semantic weight: {1.0 - BM25_WEIGHT}")

検索関数を定義しました: search_bm25, search_semantic, search_hybrid, rerank
  第1段検索 top_k: 20, 第2段 Rerank top_k: 5
  BM25 weight: 0.3, Semantic weight: 0.7


**検索機能の単体動作確認**

In [13]:
# 検索機能の単体動作確認
test_query = "高速回転ホイールの寿命試験"

print("=" * 60)
print(f"テストクエリ: {test_query}")
print("=" * 60)

# 1. BM25 キーワード検索
print("\n--- BM25 キーワード検索 ---")
bm25_results = search_bm25(test_query)
for r in bm25_results[:3]:
    print(f"  Rank {r['rank']} (score={r['score']:.4f}, chunk_id={r['chunk_id']})")
    print(f"    {r['text'][:100]}...")

# 2. セマンティック検索
print("\n--- セマンティック検索 ---")
sem_results = search_semantic(test_query)
for r in sem_results[:3]:
    print(f"  Rank {r['rank']} (score={r['score']:.4f}, chunk_id={r['chunk_id']})")
    print(f"    {r['text'][:100]}...")

# 3. ハイブリッド検索
print("\n--- ハイブリッド検索 ---")
hybrid_results = search_hybrid(test_query)
for r in hybrid_results[:3]:
    print(f"  Rank {r['rank']} (score={r['score']:.6f}, chunk_id={r['chunk_id']})")
    print(f"    {r['text'][:100]}...")

# 4. Reranker による再順位付け（ハイブリッド検索結果を入力）
print("\n--- Reranker 再順位付け ---")
reranked_results = rerank(test_query, hybrid_results)
for r in reranked_results:
    print(f"  Rank {r['rank']} (score={r['score']:.4f}, chunk_id={r['chunk_id']})")
    print(f"    {r['text'][:100]}...")

テストクエリ: 高速回転ホイールの寿命試験

--- BM25 キーワード検索 ---
  Rank 1 (score=6.3225, chunk_id=48)
    または、リテーナインスタビリティにより発
熱、摩擦増大、摩耗等が生じ、潤滑剤が劣化、
上述と同様の過程を経る。


リテーナインスタビリティにより発熱、摩擦増
大、摩耗等が生じ、潤滑剤が劣化、潤滑機能...
  Rank 2 (score=6.0841, chunk_id=15)
    3DC/DC

コンバータ



衛星側の1次電源で直接駆動する方式ではモータ駆動用の電源に DC/DC コンバー

タがない場合がある。

この場合は、1次側と2次側のグラウンドアイソレーション

...
  Rank 3 (score=6.0223, chunk_id=19)
    2は、3.2項(i)ベアリング荷重荷重(基本静定格荷重)で説明するように、ベアリングの基本静定格

荷重はベアリング内外輪間のアライメント(内輪と外輪の相対角度)に依存するため、アライメント変動

が...

--- セマンティック検索 ---
  Rank 1 (score=0.9192, chunk_id=48)
    または、リテーナインスタビリティにより発
熱、摩擦増大、摩耗等が生じ、潤滑剤が劣化、
上述と同様の過程を経る。


リテーナインスタビリティにより発熱、摩擦増
大、摩耗等が生じ、潤滑剤が劣化、潤滑機能...
  Rank 2 (score=0.9048, chunk_id=32)
    リテーナインスタビリティ、ギャッピングと予圧との相関を把握し、総合的に最適な

予圧量の設定は、今後の研究課題である。



予圧量の確認は、一般的には、あらかじめ予圧量と起動摩擦トルクの関係を把握し...
  Rank 3 (score=0.8998, chunk_id=60)
    用パターンを考慮の上、寿命要求について調整すること。



運用パターンには、回転数、ゼロクロス回数、急加減速の程度を考慮すること。



必要に応じ個別に寿命評価を行うこと。



4.2.2.  ...

--- ハイブリッド検索 ---
  Rank 1 (score=0.016393, chunk_id=48)
    または、リテ

**検索機能を LLM の tool として定義**
- ハイブリッド検索 + Reranker を LangChain の `@tool` デコレータで定義する。
- LLM がユーザの質問に対して自動的に検索ツールを呼び出し、RAG を実現する。
- LLM のコンテキスト長を圧迫しないため、検索結果を踏まえた LLM への入力を、`MAX_RETURN_CHARS` に制限している。

**`@tool`について**
以下は、LLM がツールの使い方を理解するために、LLM に参照されるテキストです。LLM に、どのようにツールを使わせたいのか、また、どのような引数（入力）が必要なツールなのかを、明示する必要があります。
```
"""
外部ナレッジベースから、クエリに関連する情報を検索・取得します。
ユーザーの質問に対し、具体的な事実、データ、あるいは詳細な文脈が必要な場合、
自身の知識だけで回答せずに必ずこのツールを使用してください。

Args:
    query: 検索したい内容を表す、具体的かつ完全な文章（日本語）。
"""
```

また、`@tool` 内では、エラーハンドリングを記述しています。tool の使用でエラーが生じても、生のログを LLM に渡すのではなく、テキスト情報でエラーを伝えることで、LLM を混乱させないことを意図しています。

In [14]:
# 検索機能を LLM の tool として定義する。
from langchain_core.tools import tool  # type: ignore

MAX_RETURN_CHARS = 8000  # LLM のコンテキストを圧迫しないための戻り値の上限文字数


@tool
def search_document(query: str) -> str:
    """外部ナレッジベースから、クエリに関連する情報を検索・取得します。
    ユーザーの質問に対し、具体的な事実、データ、あるいは詳細な文脈が必要な場合、
    自身の知識だけで回答せずに必ずこのツールを使用してください。

    Args:
        query: 検索したい内容を表す、具体的かつ完全な文章（日本語）。
    """
    try:
        hybrid_results = search_hybrid(query)
        reranked = rerank(query, hybrid_results)
    except Exception as e:
        return f"検索中にエラーが発生しました: {e}"

    if not reranked:
        return "検索結果が見つかりませんでした。"

    passages = []
    total_chars = 0
    for r in reranked:
        passage = f"[チャンク {r['chunk_id']}] (スコア: {r['score']:.4f})\n{r['text']}"
        total_chars += len(passage)
        if total_chars > MAX_RETURN_CHARS:
            break
        passages.append(passage)
    return "\n\n---\n\n".join(passages)


tools = [search_document]

print("=== RAG Tools ===")
for t in tools:
    print(f"  - {t.name}: {t.description[:80]}...")

=== RAG Tools ===
  - search_document: 外部ナレッジベースから、クエリに関連する情報を検索・取得します。
    ユーザーの質問に対し、具体的な事実、データ、あるいは詳細な文脈が必要な場合、
    ...


**動作確認**
- ReAct エージェントで検索ツールを使った RAG の動作を確認する。

In [15]:
# ReAct エージェントを構築して RAG の動作確認を行う。
from langchain.agents import create_agent  # type: ignore
from langgraph.checkpoint.memory import InMemorySaver  # type: ignore

memory = InMemorySaver()

system_prompt = """
あなたは、提供された検索ツールを使用して、ユーザの質問に回答するAIアシスタントです。
以下の制約事項を厳守してください。

1. **情報源の限定**: 
   回答は必ず `search_document` ツールで取得した情報のみに基づいて作成してください。
   あなたの事前知識や推測を含めないでください。

2. **「知らない」と言う勇気**:
   検索結果に回答に必要な情報が含まれていない場合は、無理に回答を作らず、「提供された文書にはその情報が含まれていません」と明確に伝えてください。

3. **回答フォーマット**:
   回答の最後には、必ず以下の形式で結論をまとめてください。

# 結論
- ユーザの質問: （質問の要約）
- 回答: （検索結果に基づく簡潔な回答）
"""

agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=memory,
    system_prompt=system_prompt,
)

In [16]:
# 動作確認: エージェントに質問する。
config = {"configurable": {"thread_id": "rag-test"}}

response = await agent.ainvoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "高速回転ホイールの寿命試験ではどのような結果が得られましたか？",
            }
        ]
    },
    config=config,
)

print("=== RAG Agent Result ===\n")
for msg in response["messages"]:
    if isinstance(msg.content, list):
        msg.content = "\n".join(
            item["text"] for item in msg.content if item.get("type") == "text"
        )
    msg.pretty_print()

=== RAG Agent Result ===

================================ Human Message =================================

高速回転ホイールの寿命試験ではどのような結果が得られましたか？
================================== Ai Message ==================================
Tool Calls:
  search_document (2b432ea1-e165-455a-adc6-e5f66b41fd51)
 Call ID: 2b432ea1-e165-455a-adc6-e5f66b41fd51
  Args:
    query: 高速回転ホイールの寿命試験 結果
================================= Tool Message =================================
Name: search_document

[チャンク 59] (スコア: 0.9731)
高速回転ホイール(高速回転ホイール開発を通しての知見)

                 65

表 5-1(1/2)

高速回転ホイール  仕様諸元



製造業者

最大蓄積角運動量[Nms]





最大回転数[rpm]


最大制御トルク「Nm]
ロストルク[Nm]
起動トルク[Nm]
ランアップ時間

[min.]@30

Nms
コーストダウン時間[min.]@30

Nms

消費電力[W]



電源電圧[V]


トルクスケールファクタ(トルク定数)

質量[kg]



寸法[mm]



機械環境(ランダム振動)

擾乱



アライメント(取付け面に対して)



使用温度範囲[°C]



寿命[yrs]



高速回転ホイール
(株)三菱プレシジョン
10-30(Type M)
30-80(Type L)
6000
≧0.1
≦0.03


≦0.014
≦5.5
≦14.8


Peak:135@6000rpm,0.1Nm
Steady:<33
30

~52Unregulated
0.070Nm/A

(

ノミナル

)


≦10.1(Type